In [ ]:
from aiy.vision.inference import CameraInference
from aiy.vision.models import object_detection
from aiy.vision.streaming.server import StreamingServer
from aiy.vision.streaming import svg
from aiy.leds import Leds, Color
from aiy.board import Board
from gpiozero import Servo
from aiy.pins import PIN_A

from picamera import PiCamera
from IPython.display import Image, display, clear_output

import contextlib
import time

### Animal Detector

The network we use not can detect prople, dogs and cats. Remember if your joy detector is running, you need to turn it off using the commands

```
sudo systemctl stop joy_detection_demo.service
```

The code below initializes the servo assuming you connected it to PIN_A. Ignore the notics when runnig the code. Also it **seems the only way to run this cell twice is to restart the kernel** (still debugging), so run it once and then change and play wioth the code below.

In [ ]:
servo = Servo(PIN_A)

I added some code here that is called to create the overlay - basically the box around the objects and the labels above. You can customize it and/or add information you want to overlay on the camera feed.

In [ ]:
def svg_overlay(objects, frame_size, labels):

    width, height = frame_size
    doc = svg.Svg(width=width, height=height)

    for obj in objects:
        x, y, w, h = obj.bounding_box
        doc.add(svg.Rect(x=int(x), y=int(y), width=int(w), height=int(h), rx=10, ry=10,
                         fill_opacity=0.1,style='fill:white;stroke:white;stroke-width:4px'))

        doc.add(svg.Text(labels[obj.kind], x=x, y=y-12, fill='red', font_size=50))

    return str(doc)

Some lists to set the labels and the colors we want the button to be.

In [ ]:
labels = ['Background','Person', 'Cat', 'Dog']
colors = [Color.RED, Color.BLUE, Color.PURPLE, Color.GREEN]

#### Main loop

Here is our main loop based on the code we used last time. Look at the comments on what was changed. Basically we removed the part saving the picture and adjust the servo value to the joy score each frame. We also added the streaming back in, to while this cell runs, you can connect to http://orcspi-vis.local:4664 and see the stream.

In [ ]:
with contextlib.ExitStack() as stack:
    leds   = stack.enter_context(Leds())
    camera = stack.enter_context(PiCamera(sensor_mode=4, resolution=(820, 616)))
    board  = stack.enter_context(Board())

    def match_reset():
        print('Reset ..')
        leds.update(Leds.rgb_on(Color.RED))
        servo.min()    

    # This starts and runs the streaming of the camera
    server = stack.enter_context(StreamingServer(camera))  

    # Sets what should happen when button is pressed
    board.button.when_pressed = match_reset
    
    print ("Loading model - hold on ..")
    
    # Move the servo to low (-1)
    servo.min()
    
    # Do inference on VisionBonnet
    with CameraInference(object_detection.model()) as inference:
        
        try:   
            leds.update(Leds.rgb_on(Color.RED))
            for result in inference.run():
                objects = object_detection.get_objects(result)
                
                # This sends the overlay (boxes) to add to the camera stream
                server.send_overlay(svg_overlay(objects, (result.width, result.height), labels))

                if len(objects) >= 1:
                    clear_output(wait=True)                 
                    leds.update(Leds.rgb_on(colors[objects[0].kind]))
                    
                    for i, obj in enumerate(objects):
                        print('Object #%d: %s' % (i, obj))

                    # This sets the servo to the joy score value
                    # To use the full range (-1 to 1) we multiply the score by 2 and subtract 1
                    servo.value = (2./3. * (objects[0].kind)) - 1
                                                                                  
        except KeyboardInterrupt:
            print("Interrupted ..")
            
    leds.update(Leds.rgb_off())
    
    # Servo back to the minimum upon ending
    servo.min()
    
    print("Done")